In [1]:
import os
import glob

In [2]:
print('Number of test images : ' + str(len(os.listdir('data/liver-ultrasound-detection-unzip/images/test'))))
print('Number of test images : ' + str(len(os.listdir('data/pre_hisep/images/test'))))

Number of test images : 5153
Number of test images : 5153


In [26]:
from ultralytics import YOLO

model = YOLO('runs_yoloV9e/detect/train14/weights/best.pt')

In [27]:
# results_val = model.val()

In [28]:
import pandas as pd
sample_submission = pd.read_csv('data/liver-ultrasound-detection-unzip/sample_submission.csv')
submission = sample_submission.copy()
submission

,Image File,Annotation,Label
0,18999,"[[281, 183, 339, 266]]",[1]
1,102037,"[[263, 171, 359, 263]]",[4]
2,138340,"[[85, 235, 178, 333]]",[4]
3,140589,NaN,NaN
4,20235,NaN,NaN
...,...,...,...
5148,3575,NaN,NaN
5149,190,NaN,NaN
5150,1404,NaN,NaN
5151,184,NaN,NaN


In [29]:
sample_submission.loc[~sample_submission['Annotation'].isna().values, 'Annotation']

0                             [[281, 183, 339, 266]]
1                             [[263, 171, 359, 263]]
2                              [[85, 235, 178, 333]]
24      [[270, 195, 335, 253], [268, 264, 316, 286]]
115     [[387, 268, 513, 361], [427, 203, 456, 237]]
1001                                              []
Name: Annotation, dtype: object

In [30]:
index_sample = sample_submission.loc[~sample_submission['Annotation'].isna().values, 'Annotation'].index
index_sample

Index([0, 1, 2, 24, 115, 1001], dtype='int64')

In [50]:
from tqdm.notebook import tqdm, trange
for i in trange(submission.shape[0]):
  if i in index_sample:
      continue
  results = model('data/liver-ultrasound-detection-unzip/images/test/' + str(submission.iloc[i, 0]) + '.jpg', conf=0.43, verbose=False)
  submission.loc[i,'Annotation'] = str(results[0].boxes.xyxy.round().cpu().numpy().astype(int).tolist())
  submission.loc[i,'Label'] = str(results[0].boxes.cls.cpu().numpy().astype(int).tolist())

  0%|          | 0/5153 [00:00<?, ?it/s]

In [51]:
submission

,Image File,Annotation,Label
0,18999,"[[281, 183, 339, 266]]",[1]
1,102037,"[[263, 171, 359, 263]]",[4]
2,138340,"[[85, 235, 178, 333]]",[4]
3,140589,"[[597, 375, 659, 439]]",[4]
4,20235,"[[295, 115, 341, 151]]",[1]
...,...,...,...
5148,3575,[],[]
5149,190,[],[]
5150,1404,"[[1270, 882, 1489, 1080]]",[3]
5151,184,[],[]


In [52]:
print(submission.loc[0, 'Label'] == sample_submission.loc[0, 'Label'])
print(submission.loc[1, 'Label'] == sample_submission.loc[1, 'Label'])
print(submission.loc[2, 'Label'] == sample_submission.loc[2, 'Label'])
print(submission.loc[24, 'Label'] == sample_submission.loc[24, 'Label'])
print(submission.loc[115, 'Label'] == sample_submission.loc[115, 'Label'])
print(submission.loc[1001, 'Label'] == sample_submission.loc[1001, 'Label'])

True
True
True
True
True
True


In [53]:
sum(submission['Label'] == '[]')

3583

In [54]:
sum(pd.read_csv('submission_yoloV9e39ep_conf042.csv')['Annotation'] == submission['Annotation'])

5075

In [44]:
submission.to_csv('submission_yoloV9e39ep_conf043.csv', index = False)

In [7]:
!nvidia-smi

Thu May 30 23:41:52 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:03:00.0 Off |                    0 |
| N/A   37C    P0    53W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   